In [15]:
import cv2
import numpy as np


In [14]:
# 第一步：读入图片
image_path = 'test_images/test3.jpg'
image = cv2.imread(image_path)
cv2.imshow('image', image)
cv2.waitKey(0)


-1

In [20]:
# 第二步：高斯平滑，去除噪声，canny边缘检测
def canny(img, ksize=(5,5), sigmaX=0, low_threshold=100, high_threshold=200):
	
	image = cv2.GaussianBlur(img, ksize, sigmaX)
	
	return cv2.Canny(image, low_threshold, high_threshold)

In [21]:

canny_img = canny(image)
cv2.imshow('canny_img', canny_img)
cv2.waitKey(0)


-1

In [22]:
# 第三步: 选定ROI, 对图像进行掩模操作，只显示ROI区域
def region_of_interest(img):
	height = img.shape[0]
	width = img.shape[1]
	region_of_interest_vertices = [(0, height),
	                               (width/2, height/2),
	                               (width, height)]
	mask = np.zeros_like(img)
	color = 255
	cv2.fillPoly(mask, np.array([region_of_interest_vertices], np.int32), color)
	mask_image = cv2.bitwise_and(img, mask)
	return mask_image


In [24]:
ROI_img = region_of_interest(canny_img)
cv2.imshow('ROI_img', ROI_img)
cv2.waitKey(0)

-1

In [16]:
# 第四步：hough变换，检测直线，并与原图混合
def hough_lines(img, rho=6, theta=np.pi/180, threshold=160, min_line_len=40, max_line_gap=25):
	
	lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len,
	                        maxLineGap=max_line_gap)
	
	return lines

def draw_lines(img, lines, color=[255,0,0], thickness=3):
	
	if lines is None:
		return 
	img = np.copy(img)
	img_channels = img.shape[2]
	line_img = np.zeros((img.shape[0], img.shape[1], img_channels), dtype=np.uint8)
	for line in lines:
		for x1, y1, x2, y2 in line:
			cv2.line(line_img, (x1, y1), (x2, y2), color, thickness)
	img = cv2.addWeighted(img, 0.8, line_img, 1.0, 0.0)
	return line_img, img

In [25]:
lines = hough_lines(ROI_img)
line_img, img = draw_lines(image, lines)
cv2.imshow('line_img', line_img)
cv2.imshow('img', img)
cv2.waitKey(0)

-1

In [ ]:
#第6步：直线拟合，根据端点画出左右lane
